In [2]:
import pickle
import numpy as np
import time
import pandas as pd
import seaborn as sns
from tabulate import tabulate
from evolution_utils import X_train, X_test, Y_train, Y_test, X_bow, all_models_score_table, logmodel, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
def predicted_sentiment_ratio(y_test, predictions):
    positive_percent = np.count_nonzero(predictions==1)*100//len(predictions)
    negative_percent = np.count_nonzero(predictions==0)*100//len(predictions)
    # print(f"Y_test: pos/neg percentage ~ {np.count_nonzero(y_test==1)*100//len(y_test)}, {np.count_nonzero(y_test==0)*100//len(y_test)}")
    return positive_percent, negative_percent

In [4]:
def test_accuracy(selected_genes):
    st = time.time()
    logmodel.fit(X_train[:, selected_genes], Y_train)
    et = time.time()

    predictions = logmodel.predict(X_test[:, selected_genes])
    ratio = predicted_sentiment_ratio(Y_test, predictions)
    return accuracy_score(Y_test, predictions), et-st, ratio

In [5]:
def get_intersection_genes(result_set):
    size_of_chromo = None
    common_indexes = {}
    best_chromo_in_each_run = []
    for res in result_set:
        best_chromo = res[0][-1]   
        size_of_chromo = len(best_chromo)
        chromo_features = np.where(best_chromo != 0)[0]
        best_chromo_in_each_run.append([best_chromo, chromo_features.shape[0], res[1][-1]])

        for index in chromo_features:
            common_indexes[index] = common_indexes.setdefault(index, 0) + 1
            
    sorted_common_indexes = np.array(sorted(common_indexes.items(), key=lambda x: x[1], reverse=True))
    
    return sorted_common_indexes, best_chromo_in_each_run

In [7]:
def accuracy_plot(table):
    table_df = pd.DataFrame(table)
    sns.lineplot(x='Accuracy', y='value', 
             data=pd.melt(table_df, ['Accuracy']))

# Collect Common indexes from res of n run evolution


# Final Results

In [156]:
# def tabulate_results(chromo_set, n):
#     genes = [30, 50, 60, 100, 150, 200, 250, -1]
#     accuracy = []
#     time_to_train = []
#     sentiment_ratio = []
#     for i in genes:
#         top_i_common_indexes, chromo_len = pick_top_n_genes_n_run(i, chromo_set, n)
#         top_n_genes = top_i_common_indexes[:, 0]
#         common_index_count = top_i_common_indexes[:, 1]
#         acc, ttt, sr = test_accuracy(top_n_genes)
#         accuracy.append(acc)
#         time_to_train.append(ttt)
#         sentiment_ratio.append(sr)

    
#     table_data = {'Genes': genes,
#                 'Accuracy': accuracy,
#                 'Time Taken to Train': time_to_train,
#                 'Sentiment Ratio(p/n)': sentiment_ratio
#                 }
#     print(top_n_genes)
#     print('Actual chromosome length in X_Test: ', X_test.shape[1])
#     print('RandomForest with full length -', all_models_score_table.iloc[0, 1:])
#     print('Total Common genes count ~(-1): ', top_i_common_indexes)
#     print(chromo_len)
#     print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))
#     return table_data, top_n_genes, top_i_common_indexes


In [161]:
def tabulate_results(chromo_set):
    print('Actual chromosome length in X_Test: ', X_bow.shape[1])
    print('\nRandomForest with full length -', all_models_score_table.iloc[0, 1:], "\n")
    # print('Total Common genes count ~(-1): ', top_i_common_indexes)

    accuracy = []
    features_count = []
    time_to_train = []
    sentiment_ratio = []

    sorted_common_indexes, best_chromo_in_each_run = get_intersection_genes(chromo_set)
    common_indexes_count = sorted_common_indexes[:, 1]
    # print("Total common indexes:", sorted_common_indexes.shape[0])

    count_thresholds = list(set(common_indexes_count))[::-1]
    print("Intersection frequency:", count_thresholds, "\n")

    for i in count_thresholds:
        selected_features  = sorted_common_indexes[np.where(sorted_common_indexes[:, 1] >= i), 0][0]
        features_count.append(len(selected_features))
        acc, ttt, sr = test_accuracy(selected_features)
        accuracy.append(acc)
        time_to_train.append(ttt)
        sentiment_ratio.append(sr)

    
    table_data = {'Count Threshold': count_thresholds,
                'Feature count': features_count,
                'Accuracy': accuracy,
                'Time Taken to Train': time_to_train,
                'Sentiment Ratio(p/n)': sentiment_ratio
                }
    print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))
    return table_data, best_chromo_in_each_run


In [8]:
with open('pickles/ga/n_run_az_100fc.pkl', 'rb') as rf:
    n_ga_az_100fc = pickle.load(rf)
    n_ga_az_100fc_table, best_chromos_co = tabulate_results(n_ga_az_100fc)
    n_ga_az_100fc_table

FileNotFoundError: [Errno 2] No such file or directory: 'pickles/ga/n_run_az_100fc.pkl'

In [155]:
with open('pickles/kbga/amazon/n_run_az_kbga.pkl', 'rb') as rf:
    n_kbga = pickle.load(rf)
    n_run_kbga_table = tabulate_results(n_kbga, 30)
    n_run_kbga_table

Actual chromosome length in X_Test:  1566
RandomForest with full length - Accuracy             0.796
Exec_Time_secs    1.323454
Name: 0, dtype: object
Total Common genes count ~(-1):  [[ 58   9]
 [116   9]
 [868   9]
 ...
 [239   1]
 [263   1]
 [890   1]]
[100 100 100 100 327 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 190 100 100 100 100]
┌───────────────────┬────────────┬───────────────────────┬────────────────────────┐
│   Count Threshold │   Accuracy │   Time Taken to Train │ Sentiment Ratio(p/n)   │
├───────────────────┼────────────┼───────────────────────┼────────────────────────┤
│                 1 │      0.676 │               1.00728 │ (54, 45)               │
├───────────────────┼────────────┼───────────────────────┼────────────────────────┤
│                 2 │      0.664 │               1.53589 │ (54, 46)               │
├───────────────────┼────────────┼───────────────────────┼────────────────────────┤
│                 3 │      0.676 